In [1]:
!pip install gradio transformers bitsandbytes accelerate peft trl pandas datasets

In [2]:
!pip install transformers

In [3]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread
import pandas as pd
import bitsandbytes

In [6]:
!pip install --upgrade pandas datasets transformers

import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

# Configuration
dataset_id = "jojogo9/Food_Recipes"
model_id = "eaglet/Testing"
output_model = "llm-007"
start_row = 600  # Starting row
end_row = 650  # Ending row
sample_size = 50  # Sample size for the dataset
max_length = 128  # Max length for tokenization

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# !pip install --upgrade pandas datasets transformers

# import torch
# from datasets import load_dataset, Dataset
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Configuration
# dataset_id = "jojogo9/Food_Recipes"
# model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# output_model = "llm-007"
# start_row = 150  # Starting row
# end_row = 250  # Ending row
# sample_size = 100  # Sample size for the dataset
# max_length = 128  # Max length for tokenization

# # Check device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to prepare training data with start and end row parameters
def prepare_train_data(data_id, start_row=0, end_row=None, sample_size=50):
    # Load the dataset
    data = load_dataset(data_id, split="train")
    data_df = data.to_pandas()

    # Select the rows between start_row and end_row
    if end_row is not None:
        data_df = data_df.iloc[start_row:end_row]
    else:
        data_df = data_df.iloc[start_row:]

    # If the resulting dataset is larger than sample_size, sample the data
    if len(data_df) > sample_size:
        data_df = data_df.sample(n=sample_size, random_state=42)

    # Format the text column as required
    data_df["text"] = data_df.apply(
        lambda x: (
            "im_start user\n" + x["ingredients"] + "\nim_end\n" +
            "im_start assistant\n" + x["name"] + "\nim_end\n" +
            "im_start steps: " + x["steps"] + "\nim_end\n"
        ),
        axis=1
    )

    # Convert the DataFrame back to a Dataset
    data = Dataset.from_pandas(data_df)

    return data

# Prepare the training data
data = prepare_train_data(dataset_id, start_row=start_row, end_row=end_row, sample_size=sample_size)

# Function to get model and tokenizer
def get_model_and_tokenizer(model_id):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # load_in_8bit=True  # If 4-bit isn't supported directly
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

# Load model and tokenizer
model, tokenizer = get_model_and_tokenizer(model_id)

# Function to tokenize, encode, and truncate the dataset
def encode_and_truncate_dataset(dataset, tokenizer, max_length=128):
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",  # Pad to the maximum length
            truncation=True,       # Truncate to the maximum length
            max_length=max_length  # Maximum length of sequences
        )

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    return tokenized_dataset

# Encode and truncate the dataset
encoded_dataset = encode_and_truncate_dataset(data, tokenizer, max_length)

# Example to check the first encoded item
print(encoded_dataset[0])

# Save the dataset if needed
encoded_dataset.save_to_disk("encoded_dataset")

# Now your encoded_dataset is ready for training with the language model


adapter_config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

{'name': 'cream  of mushroom soup', 'id': 51922, 'minutes': 10, 'contributor_id': 59064, 'submitted': '2003-01-21', 'tags': "['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'healthy', 'bisques-cream-soups', 'soups-stews', 'vegetables', 'easy', 'no-cook', 'vegan', 'vegetarian', 'food-processor-blender', 'dietary', 'low-cholesterol', 'low-carb', 'mushrooms', 'healthy-2', 'low-in-something', 'equipment', 'small-appliance', '3-steps-or-less', 'technique']", 'nutrition': '[472.9, 64.0, 9.0, 38.0, 28.0, 19.0, 5.0]', 'n_steps': 3, 'steps': "['place walnuts , garlic water , nama shoyu , and rosemary in blender and blend until creamy', 'add ground flax seed and blend well', 'pour into a serving dish and add mushrooms']", 'description': "if you are not a raw food vegan then try this recipe with an open mind. raw foodists try not to cook anything if they can avoid it. i haven't tried this recipe but posted it because it looked interesting.", 'ingredients': "['wa

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

# Define your LoraConfig
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Define a function to get model and tokenizer
def get_model_and_tokenizer(model_id):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # Remove quantization_config argument
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Example usage
model_id = 'eaglet/Testing'
model, tokenizer = get_model_and_tokenizer(model_id)


In [9]:
# Define the training arguments
training_arguments = TrainingArguments(
    output_dir="output_model",
    per_device_train_batch_size=2,  # Reduce batch size 8
    gradient_accumulation_steps=32,   # Increase accumulation steps 8
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=3,
    max_steps=50, #PUT 250 LATER
    fp16=True,
    # push_to_hub=True
)

# Assume 'data' is your training dataset
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,0.658500
20,0.573400
30,0.541600
40,0.577000
50,0.491400


TrainOutput(global_step=50, training_loss=0.5683904361724853, metrics={'train_runtime': 200.7635, 'train_samples_per_second': 15.939, 'train_steps_per_second': 0.249, 'total_flos': 2588508717170688.0, 'train_loss': 0.5683904361724853, 'epoch': 28.16})

In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=256,

In [11]:
trainer.evaluate(encoded_dataset)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


{'eval_loss': 1.1202363967895508,
 'eval_runtime': 1.3108,
 'eval_samples_per_second': 38.145,
 'eval_steps_per_second': 5.34,
 'epoch': 28.16}

In [13]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch

import os

model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype=torch.float16,load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)
model_path = "/content/output_model/checkpoint-50"
# model_path = "/recipe-generator-G97"

peft_model = PeftModel.from_pretrained(model,model_path,from_transformers=True,device_map="auto")
model = peft_model.merge_and_unload()

In [14]:
!pip install huggingface_hub --q
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [15]:
# model.push_to_hub("eaglet/Testing")
# tokenizer.push_to_hub("eaglet/Testing")
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import HfApi, HfFolder, Repository

# Step 1: Load your fine-tuned model and tokenizer
model_name = "/content/output_model/checkpoint-50"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Step 2: Create a repository on Hugging Face
api = HfApi()
token = HfFolder.get_token()
repo_id = "eaglet/Testing"  # Replace with your details

# api.create_repo(repo_id=repo_id, token=token)

# Step 3: Clone the repository and push the model and tokenizer
repo = Repository(local_dir="/content/local", clone_from=repo_id)

# Save the model and tokenizer locally
model.save_pretrained("/content/local")
tokenizer.save_pretrained("/content/local")

# Push the model and tokenizer to the Hugging Face Hub
repo.push_to_hub(commit_message="Initial commit")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/eaglet/Testing into local empty directory.


Download file adapter_model.safetensors:   0%|          | 17.5k/4.31M [00:00<?, ?B/s]

Download file tokenizer.model:   6%|5         | 27.1k/488k [00:00<?, ?B/s]

Clean file tokenizer.model:   0%|          | 1.00k/488k [00:00<?, ?B/s]

Clean file adapter_model.safetensors:   0%|          | 1.00k/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


Upload file adapter_model.safetensors:   1%|          | 32.0k/4.31M [00:00<?, ?B/s]

To https://huggingface.co/eaglet/Testing
   0e23ded..2c61a5c  main -> main

   0e23ded..2c61a5c  main -> main



'https://huggingface.co/eaglet/Testing/commit/2c61a5c77f048e953903b80add734117e266de14'

In [ ]:
# from huggingface_hub import HfApi, Repository

# # Replace with your Hugging Face username and model name
# username = "eaglet"
# model_name = "JulyTesting"

# # Initialize the API
# api = HfApi()

# # Clone the existing repository
# # api.upload_folder(
# #     repo_id=f"{username}/{model_name}",
# #     folder_path="/content/output_model/checkpoint-250",
# #     commit_message="Updated the fine-tuned model"
# # )
# api.upload_folder(
#     repo_id=f"{username}/{model_name}",
#     folder_path="/content/output_model/checkpoint-250",
#     path_in_repo="checkpoint-250"
# )

In [16]:
def formated_prompt(quest):
  return f"<user>user:\n{quest}\n<assistant>assistant:\n"

In [23]:
# from transformers import GenerationConfig
# from time import perf_counter

# def generate_resp(user_input):
#   prompt = formated_prompt(user_input)
#   inputs = tokenizer([prompt], return_tensors="pt")
#   generation_config = GenerationConfig(penalty_alpha=0.6,do_sample=True,
#                                          max_new_tokens=1024,pad_token_id=tokenizer.eos_token_id)
#   start_time = perf_counter()
#   inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
#   outputs = model.generate(**inputs,generation_config=generation_config)
#   # print(tokenizer.decode(outputs[0],skip_special_tokens=True))
#   output_time = perf_counter() - start_time
#   print("Time taken ",round(output_time,2))
#   return tokenizer.decode(outputs[0],skip_special_tokens=True)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from time import perf_counter

model_name = "/content/output_model/checkpoint-50"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def generate_resp(user_input):
    # Format the prompt
    prompt = formated_prompt(user_input)

    # Tokenize and move inputs to the correct device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Define the generation configuration
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        max_new_tokens=2048,
        pad_token_id=tokenizer.eos_token_id
    )

    # Start timing
    start_time = perf_counter()

    # Generate the response
    with torch.no_grad():  # Disable gradient calculations for inference
        outputs = model.generate(**inputs, generation_config=generation_config)

    # Calculate the time taken
    output_time = perf_counter() - start_time
    print("Time taken", round(output_time, 2))

    # Decode and return the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)




In [28]:
# Example usage
print(generate_resp(user_input="give an output of 5 different recepies (with short name) and its respective ingredient list and also its respective steps to prepare the dish (point wise).also give the type of cuisine (eg asian, thai etc). the given ingreidents are chicken, spices, sauces, cheese, bread"))

Time taken 27.34
<user>user:
give an output of 5 different recepies (with short name) and its respective ingredient list and also its respective steps to prepare the dish (point wise).also give the type of cuisine (eg asian, thai etc). the given ingreidents are chicken, spices, sauces, cheese, bread
<assistant>assistant:
here you go:
recipes - chicken satay on bun
recipe:
ingredients:
oil in wok, pan
1 / 2 cup of vegetable oil
6 - 7 chicken strips, sliced
about 2 tablespoons of kuk lak sauce
2 tablespoons kuk lak ketchup
4 tablespoons rice vinegar
1 tablespoon honey
1 tablespoon sugar
2 tablespoons mushroom soup
2 tablespoons sliced green onions
vegetable sticks or rolls
sliced cucumber and pickled ginger (optional)
dipping sauce

steps:
1. in a wok pan over medium high heat , add the oil
2. stir-fry the chicken strips , turning occasionally with a wooden spoon
3. in another bowl add 1 / 4 cup of kuk lak sauce , kuk lak ketchup and mushroom soup
4. let it cook for a minute or 2 and the